In [3]:
%pip install -q datasets evaluate ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -q wandb==0.17.2 matplotlib setuptools scikit-learn

Note: you may need to restart the kernel to use updated packages.


If running this notebook in Colab, please ensure that your Hugging Face `HF_TOKEN` and your Weights & Biases `WANDB_API_KEY` are added to your Colab secrets.

Alternatively, please login to Hugging Face and Weights & Biases by running the following two cells.

In [3]:
# !huggingface-cli login

In [4]:
# !wandb login

In [1]:
import os
import random
import numpy as np
import torch

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [2]:
from datasets import load_dataset

iris = load_dataset("scikit-learn/iris")
iris

DatasetDict({
    train: Dataset({
        features: ['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species'],
        num_rows: 150
    })
})

In [3]:
features = iris['train'].features
features

{'Id': Value(dtype='int64', id=None),
 'SepalLengthCm': Value(dtype='float64', id=None),
 'SepalWidthCm': Value(dtype='float64', id=None),
 'PetalLengthCm': Value(dtype='float64', id=None),
 'PetalWidthCm': Value(dtype='float64', id=None),
 'Species': Value(dtype='string', id=None)}

In [4]:
iris.set_format("pandas")
iris_df = iris['train'][:]
iris_df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
iris_df['Species'].value_counts()

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

In [6]:
label2id = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
iris_df['Species'] = [label2id[species] for species in iris_df['Species']]
iris_df['Species'].value_counts()

Species
0    50
1    50
2    50
Name: count, dtype: int64

In [7]:
iris_df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [8]:
iris_df = iris_df.sample(frac=1, replace=False, random_state=42).reset_index(drop=True)
iris_df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,74,6.1,2.8,4.7,1.2,1
1,19,5.7,3.8,1.7,0.3,0
2,119,7.7,2.6,6.9,2.3,2
3,79,6.0,2.9,4.5,1.5,1
4,77,6.8,2.8,4.8,1.4,1
...,...,...,...,...,...,...
145,72,6.1,2.8,4.0,1.3,1
146,107,4.9,2.5,4.5,1.7,2
147,15,5.8,4.0,1.2,0.2,0
148,93,5.8,2.6,4.0,1.2,1


In [9]:
X = iris_df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
y = iris_df['Species'].values
X.shape, y.shape

((150, 4), (150,))

In [10]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
X_train_full.shape, X_test.shape, y_train_full.shape, y_test.shape

((135, 4), (15, 4), (135,), (15,))

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1, stratify=y_train_full, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((121, 4), (14, 4), (121,), (14,))

In [12]:
X_means, X_stds = X_train.mean(axis=0), X_train.std(axis=0)
X_means, X_stds

(array([5.82479339, 3.02809917, 3.7214876 , 1.18264463]),
 array([0.82193895, 0.43980145, 1.75476714, 0.75434583]))

In [13]:
X_train = (X_train - X_means) / X_stds
X_valid = (X_valid - X_means) / X_stds
X_test = (X_test - X_means) / X_stds
X_train.shape, X_valid.shape, X_test.shape

((121, 4), (14, 4), (15, 4))

In [14]:
from torch.utils.data import Dataset, DataLoader

class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [15]:
train_set = IrisDataset(X_train, y_train)
len(train_set)

121

In [16]:
# Sanity check:
train_set[0]

(tensor([ 1.0648, -1.2008,  1.1845,  0.8184]), tensor(2))

In [17]:
valid_set = IrisDataset(X_valid, y_valid)
test_set = IrisDataset(X_test, y_test)
len(valid_set), len(test_set)

(14, 15)

In [18]:
batch_size = 8
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
len(train_loader)

15

In [19]:
# Sanity check:
x_batch, y_batch = next(iter(train_loader))
x_batch.shape, y_batch.shape

(torch.Size([8, 4]), torch.Size([8]))

In [20]:
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
len(valid_loader), len(test_loader)

(2, 2)

In [ ]:
# If you modified the futhark model, you will need to restart the kernel
# and re-run the notebook to get the updated model.

!rm -rf _model.* model.c model.h
!futhark multicore --library model.fut
!build_futhark_ffi model

In [21]:
import numpy as np
import _model
from futhark_ffi import Futhark

model = Futhark(_model)

In [22]:
import sys
sys.path.append("..")
from deeplearning_utils import parse_spec

In [23]:
# Get futhark model weights specification
spec = parse_spec(model.from_futhark(model.specs()))
spec

{'fc1.weight': {'tpe': 'f32', 'dims': [5, 4]},
 'fc1.bias': {'tpe': 'f32', 'dims': [5]},
 'fc2.weight': {'tpe': 'f32', 'dims': [3, 5]},
 'fc2.bias': {'tpe': 'f32', 'dims': [3]}}

In [24]:
from tqdm.autonotebook import tqdm, trange

def train_epoch(ws):
    #model.train()
    train_loss = 0
    #ws = mnist.init_weights()
    #TODO fixme to group batches as much as possible
    for x_batch, y_batch in tqdm(train_loader, desc="Training"):
        x_batch = x_batch.numpy()
        y_batch = y_batch.numpy()
        (ws, loss) = model.train(ws, batch_size, x_batch, y_batch)
        train_loss += loss
    train_loss /= len(train_set)
    train_loss = round(train_loss, 4)
    return (ws, train_loss)

In [25]:
def validate_epoch(ws):
    valid_loss = 0
    valid_acc = 0
    #TODO fixme to group batches as much as possible
    for x_batch, y_batch in tqdm(valid_loader, desc="Validation"):
        x_batch = x_batch.numpy()
        y_batch = y_batch.numpy()
        (acc, loss) = model.validate(ws, x_batch, y_batch)
        valid_loss += loss
        valid_acc += acc
    valid_loss /= len(valid_set)
    valid_loss = round(valid_loss, 4)
    valid_acc /= len(valid_set)/batch_size
    valid_acc = round(valid_acc, 4)
    return valid_loss, valid_acc

In [26]:
import wandb

n_epochs = 100

wandb_config = {
    'architecture': "MLP",
    'dataset': "Iris",
    'batch_size': batch_size,
    'learning_rate': 0.01,
    'n_epochs': n_epochs,
}
wandb.init(
    project="futhark-mlp-iris",
    config=wandb_config,
    notes="Logging min of `train_loss` & `valid_loss`, and max of `accuracy`."
)
wandb.define_metric("train_loss", summary="min")
wandb.define_metric("valid_loss", summary="min")
wandb.define_metric("accuracy", summary="max")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: leonard-schneider. Use `wandb login --relogin` to force relogin
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core


In [27]:
from deeplearning_utils import push_to_HF_hub

best_epoch = 0
best_acc = 0
ws = model.default_weights()
for epoch in trange(n_epochs, desc="Epoch"):
    (ws, train_loss) = train_epoch(ws)

    valid_loss, acc = validate_epoch(ws)

    wandb.log({'train_loss': train_loss, 'valid_loss': valid_loss, 'accuracy': acc})

    tqdm.write(f"Epoch: {epoch}, Training Loss: {train_loss}, Validation Loss: {valid_loss}, Accuracy: {acc}")

    if acc > best_acc:
        best_epoch = epoch
        best_acc = acc
        push_to_HF_hub(
            model,
            "leoschneider/futhark-mlp-iris",
            "model.fut",
            ws,
            commit_message=f"epoch: {epoch}, accuracy: {acc}",
            token=os.environ["HF_TOKEN"]
        )
wandb.finish()
print("---")
print("Done!")
print(f"Best Epoch: {best_epoch}, Best Accuracy: {best_acc}")

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/15 [00:00<?, ?it/s]

Validation:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 0, Training Loss: 1.2037, Validation Loss: 1.2295, Accuracy: 2.8571


Training:   0%|          | 0/15 [00:00<?, ?it/s]

...

Training:   0%|          | 0/15 [00:00<?, ?it/s]

Validation:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 99, Training Loss: 0.3614, Validation Loss: 0.3708, Accuracy: 6.2857


wandb: WARNING Unable to render Widget, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core


---
Done!
Best Epoch: 12, Best Accuracy: 6.2857


In [32]:
# Load model from HF hub, if you want to skip training
from deeplearning_utils import load_weights_from_hf_hub

ws = load_weights_from_hf_hub(
    model,
    "sadhaklal/mlp-iris",
    "model.safetensors",
    token=os.environ["HF_TOKEN"]
)

Loading fc1.weight with shape [5, 4] and dtype f32
[[-0.10955179  0.10089535 -0.24342752  0.29364133]
 [ 0.33094615 -0.54045284  1.4722185   0.9810878 ]
 [ 0.66089314 -0.64838004  0.8264993   0.6493943 ]
 [ 0.33597267  0.1236271  -0.30570328  0.05329869]
 [-0.11930685 -0.36874515  1.6072415   1.9744782 ]]
Loading fc1.bias with shape [5] and dtype f32
[-0.3946851  -0.83004653  2.2240942  -0.29805657 -1.019051  ]
Loading fc2.weight with shape [3, 5] and dtype f32
[[-0.44170353 -0.21288367 -2.118905    0.38234478 -0.5156934 ]
 [-0.14521228 -0.7494736   1.6898736   0.30908802 -1.2574942 ]
 [-0.14103918  1.7627541  -0.00243688  0.20337397  2.2957842 ]]
Loading fc2.bias with shape [3] and dtype f32
[ 3.2907965  -0.76823306 -2.2013984 ]


In [28]:
from tqdm.autonotebook import tqdm
import evaluate

metric = evaluate.load("accuracy")
for x_batch, y_batch in test_loader:
    x_batch, y_batch = x_batch.numpy(), y_batch.numpy()
    logits = model.from_futhark(model.predict(ws, x_batch))
    preds = logits.argmax(axis=-1)
    metric.add_batch(predictions=preds, references=y_batch)
computed_metric = metric.compute()
acc = round(computed_metric['accuracy'], 4)
print(f"Test Set Accuracy: {acc}")

Test Set Accuracy: 0.9333
